# Content-based Recommendation System
## Based on Dario Radečić tutorial
### Coded by Rebeca Bivar - DB: Armazem Paraíba

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics.pairwise import cosine_similarity 

In [2]:
db = pd.read_excel('dados_vendas.xlsx')

db.drop(["NACIMENTO", "COD_FILIAL", "ORCAMENTO", "DATA_COMPRA", "HORA", "SEXO","IDADE", "CANAL"], axis=1, inplace=True)

db.head()

#db.to_csv('contentBased.csv', sep=';')

,COD_CLIENTE,COD_PRODUTO,NOME_PRODUTO,CLASSIFICACAO,QUANTIDADE
0,05090301,25728,SMARTPHONE SAMSUNG GAL J4 CORE 16GB PRETO,CELULAR SMARTPHONE SMARTPHONE ...,1
1,05190001,25172,SMARTPHONE LG K9 TV LM-X210BMW PRETO,CELULAR SMARTPHONE SMARTPHONE ...,1
2,99DL9N01,26243,SUPORTE UNIV. BEDIN BU-25,TECNOLOGIA TV E VIDEO SUPORTE PARA TV ...,1
3,99DL9N01,26243,SUPORTE UNIV. BEDIN BU-25,TECNOLOGIA TV E VIDEO SUPORTE PARA TV ...,1
4,05992701,26574,SMARTPHONE SAMSUNG GAL A30S 64GB BRANCO,CELULAR SMARTPHONE SMARTPHONE ...,1


### Table we're going to use to check correlation between products
   - Every client id as a row
   - Every product name as a column
   - Purchase frequency by each client per item as the intersection of row and column

In [3]:
matrix = db.pivot_table(index = 'COD_CLIENTE',
                       columns = 'NOME_PRODUTO',
                       values = 'QUANTIDADE')


NOME_PRODUTO,ACESSORIO MONOPOD PAU DE SELFIE CABO AMARELO,ACESSORIO MONOPOD PAU DE SELFIE CABO AZUL,ACESSORIO MONOPOD PAU DE SELFIE CABO BRANCO,ACESSORIO MONOPOD PAU DE SELFIE CABO PRETO,ACESSORIO MONOPOD PAU DE SELFIE CABO ROSA,ALCATEL OT 236 PTO 3 CHIPS CLARO,ALTO FALANTE TS1331,ALTO FALANTE TS1760 BR,ALTO FALANTE TS6960BR PIONEER,ANTENA + RECEPTOR SKY PRE PAGO 60CM PT/CZ,...,"TELEVISOR SMART 55"" 4K UN55NU7400 SAMSUNG","TELEVISOR SMART 55"" 55P65US 4K SEMP TCL","TELEVISOR SMART 55"" 55P8M 4K UHD BLT SEMP TCL","TELEVISOR SMART 55"" P6US ULTRA HD 4K HDR SEMP","TELEVISOR SMART 55"" UHD BT UN55RU7100 SAMSUNG","TELEVISOR SMART 65"" 4K UN65NU7100 SAMSUNG","TELEVISOR SMART 65"" UHD BT UN65RU7100 SAMSUNG","TELEVISOR SMART 75"" UHD BT UN75RU7100 SAMSUNG",TV LED SEMP DL4844 F,VIVOBOOK 2G 500 CEL TOUCH X200MA CT138H
COD_CLIENTE,,,,,,,,,,,,,,,,,,,,,
00000301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00001501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00002001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00002701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00003101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Fetching recommendations
   - Calculate the correlation of de product with othe products
   - Create a df to store products with correlations 
   - Merge new df with original one and drop duplicates, having only product name, correlation value and purchase frequency
   - Sort by correlation in descending order
   - Filter products with low number of purchase (these products have not been popular with the clients)
   - Return top n correlated products

In [42]:
original = db.copy()

def get_cor_similar(matrix, product_name):
    similar = matrix.corrwith(matrix[product_name])
    corr_similar = pd.DataFrame(similar, columns=['correlation'])
    corr_similar.dropna()
    
    return corr_similar

def get_cor_product(corr_similar, original):
    cor_with_product = pd.merge(left = corr_similar,
                               right = original,
                               on='NOME_PRODUTO')[['NOME_PRODUTO', 'correlation',
                                             'QUANTIDADE']]
    
    cor_with_product.drop_duplicates().reset_index(drop=True)
    
    return cor_with_product

def get_similar_products(cor_with_product, purchase_freq_filter = 5, n_recom = 10):
       
    result = cor_with_product[cor_with_product['QUANTIDADE'] > purchase_freq_filter].sort_values(by='correlation', ascending = False)
    
    return result.head(n_recom)



In [39]:
corSimilar = get_cor_similar('TABLET NB736 MINI MS50G PRETO                ')
corProduct = get_cor_product(corSimilar, original)
get_similar_products(corProduct)

/home/rbvr/venv/lib/python3.6/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/home/rbvr/venv/lib/python3.6/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,NOME_PRODUTO,correlation,QUANTIDADE
195813,SMARTPHONE SAMSUNG GAL J4 CORE 16GB COBRE,1.0,10
206189,SMARTPHONE SAMSUNG GAL J4 CORE 16GB PRETO,1.0,5
268413,TABLET NB736 MINI MS50G PRETO,1.0,10
268195,TABLET NB733 MINI MS50X AZUL/PRETO,1.0,4
210835,SMARTPHONE SAMSUNG GAL J4 CORE 16GB PRETO,1.0,5
195884,SMARTPHONE SAMSUNG GAL J4 CORE 16GB COBRE,1.0,10
206190,SMARTPHONE SAMSUNG GAL J4 CORE 16GB PRETO,1.0,5
207443,SMARTPHONE SAMSUNG GAL J4 CORE 16GB PRETO,1.0,10
206143,SMARTPHONE SAMSUNG GAL J4 CORE 16GB PRETO,1.0,10
206081,SMARTPHONE SAMSUNG GAL J4 CORE 16GB PRETO,1.0,10
